In [24]:
import requests
from bs4 import BeautifulSoup
from math import ceil, nan
import json, re, os

headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36"}

def render_html(url):
    req = requests.get(url,headers=headers)
    page = BeautifulSoup(req.text,"html.parser")
    return page

# find product calories
def find_calories(page):
    div = page.find("div",class_="c-offer__nutrition")
    netto = div.find("div",class_="title").get_text()
    netto = re.search("(?<=\()(.*?)(?=\))",netto).group()
    table = div.find("table")
    for tag in table.find_all("tr"):
        if "kcal" in tag.text:
            cal = int(tag.find_all("td")[-1].get_text())
    if cal != 0:
        calories = f"{netto} = {cal}"
        return calories
    else:
        return None

In [25]:
url = "https://www.spar.nl/smaakt-3305554/"
page = render_html(url)

In [ ]:
# from pprint import pprint

# for script in page.find_all("script",{"type":"application/ld+json"}):
#     product = json.loads(script.get_text())
#     type_ = product['@type']
#     if type_ == "Product":
#         product_data['code'] = product['gtin13']
#         product_data['image_url'] = product['image']
#         product_data['small_image_url'] = product['image']
#         product_data['product_name'] = product['name']
#         product_data['brand'] = product['brand']
#     elif type_ == "BreadcrumbList":
#         categories = []
#         for item in product['itemListElement']:
#             categories.append(item['item']['name'])
#         product_data['categories'] = categories

In [63]:
product_data = {
    "code" : None,
    "sources" : "src",
    "image_url" : None,
    "small_image_url" : None,
    "product_name" : None,
    "brand" : None,
    "nutriscore" : None,
    "categories" : None,
    "ingredients_text" : None,
    "calories" : None,
    "it_has_nutriscore": None
}

# find product data
for script in page.find_all("script",{"type":"application/ld+json"}):
    product = json.loads(script.get_text())
    type_ = product['@type']
    if type_ == "Product":
        product_data['code'] = product['gtin13']
        product_data['image_url'] = product['image']
        product_data['small_image_url'] = product['image']
        product_data['product_name'] = product['name']
        product_data['brand'] = product['brand']
    elif type_ == "BreadcrumbList":
        categories = []
        for item in product['itemListElement']:
            categories.append(item['item']['name'])
        product_data['categories'] = categories

# find product calories
try: calories = find_calories(page)
except: calories = None

# find product ingredients
try: ingredients = page.find("div",class_="c-offer__ingredients-body").get_text().strip()
except: ingredients = None

html = str(page)
if "nutriscore" in html:
    it_has_nutriscore = True
else:
    it_has_nutriscore = False

# assign all the retrive data
product_data['calories'] = calories
product_data['ingredients_text'] = ingredients
product_data['it_has_nutriscore'] = it_has_nutriscore
        
product_data

{'code': '8719153022389',
 'sources': 'src',
 'image_url': 'https://media.spar.nl/productdetail/smaakt-200-Gram-3305554-138062.jpg',
 'small_image_url': 'https://media.spar.nl/productdetail/smaakt-200-Gram-3305554-138062.jpg',
 'product_name': 'Smaakt',
 'brand': 'Smaakt',
 'nutriscore': None,
 'categories': ['ontbijt, beleg, bakken, granen',
  'ontbijtgranen',
  'ontbijtverrijkers',
  'Smaakt'],
 'ingredients_text': "TARWEzemelen*. *=biologisch Kan sporen bevatten van andere glutenbevattende granen, pinda's, selderij, sesamzaad.",
 'calories': None,
 'it_has_nutriscore': False}

In [ ]:
# import random, json

# random.choice(json.load(open("./urls/product_urls.json")))